In [1]:
from wmfdata import hive, spark

### Observations
editattemptstep: event.session_token = mediawiki_edit_attempt: performer.session_id

## Compare metrics between editattemptstep and event.mediawiki_edit_attempt

First, we check a few events at the hour level. Here I am looking at visualeditor events

In [11]:
name_query1=spark.run('''
SELECT 
  DISTINCT name, count(1) AS ve_events
  FROM event.mediawiki_edit_attempt
  WHERE name like 'eas.ve%'
  AND year = 2022
  AND month= 11
  AND day = 3
  AND hour = 4
  AND normalized_host.project = 'test'
  GROUP BY name 
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [12]:
name_query1.sort_values(by=['ve_events','name'],ascending=False)

,name,ve_events
5,eas.ve.ready,17
7,eas.ve.loaded,17
3,eas.ve.init,17
2,eas.ve.save_success,15
4,eas.ve.save_intent,15
6,eas.ve.save_attempt,15
0,eas.ve.first_change,15
1,eas.ve.abort,2


In [15]:
action_query1=spark.run('''
SELECT 
  DISTINCT event.action AS action, count(1) AS ve_events
  FROM event.editattemptstep
  WHERE event.editor_interface= 'visualeditor'
  AND year = 2022
  AND month= 11
  AND day = 3
  AND hour = 4
  AND wiki = 'testwiki' 
  GROUP BY action 
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


This matches exactly with the events from mediawiki_edit_attempt

In [16]:
action_query1.sort_values(by=['ve_events','action'],ascending=False)

,action,ve_events
2,ready,17
1,loaded,17
3,init,17
5,saveSuccess,15
6,saveIntent,15
0,saveAttempt,15
7,firstChange,15
4,abort,2


Next I compared events from 2022-10-09 hour "8" 

In [84]:
name_query2=spark.run('''
SELECT 
  DISTINCT name, count(1) AS wikitext_events
  FROM event.mediawiki_edit_attempt
  WHERE name like 'eas.mf%'
  AND year = 2022
  AND month= 10
  AND day = 9
  AND hour = 8
  AND normalized_host.project = 'test'
  GROUP BY name 
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [85]:
name_query2.sort_values(by=['wikitext_events','name'],ascending=False)

,name,wikitext_events
0,eas.mf.ready,5
6,eas.mf.loaded,5
1,eas.mf.init,5
3,eas.mf.save_success,2
2,eas.mf.save_intent,2
5,eas.mf.save_attempt,2
4,eas.mf.first_change,2


In [39]:
action_query2=spark.run('''
SELECT 
  DISTINCT event.action AS action, count(1) AS wikitext_events
  FROM event.editattemptstep
  WHERE event.editor_interface = 'wikitext'
  AND year = 2022
  AND month= 10
  AND day = 9
  AND hour = 8 
  AND wiki = 'testwiki' 
  GROUP BY action 
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


Note that the number of events captured in the same "hour" is different in both tables.

In [40]:
action_query2.sort_values(by=['wikitext_events','action'],ascending=False)

,action,wikitext_events
2,ready,1
1,loaded,1
0,init,1


In [74]:
eas_query1=spark.run('''
SELECT 
  *
  FROM event.editattemptstep
  WHERE year = 2022
  AND month= 10
  AND day = 9
  AND hour = 8 
  AND wiki = 'testwiki'  
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [75]:
eas_query1

,dt,event,ip,recvfrom,revision,schema,seqid,useragent,uuid,webhost,...,_schema,client_dt,http,meta,user_agent_map,normalized_host,year,month,day,hour
0,2022-10-09T08:45:38.473Z,"(init, ecc907e1cd3cc9b88ac4, wikitext, click, ...",None,None,None,EditAttemptStep,None,"(Chrome, 106, None, Lenovo TB-7104F, False, Fa...",None,test.wikipedia.org,...,None,2022-10-09T08:45:44.524Z,"(None, None, None, None, {'user-agent': 'Mozil...","(None, 195de194-b0e5-4239-81e2-d9e99ac6f282, 2...","{'os_family': 'Android', 'browser_major': '106...","(wikipedia, test, [], org, wikipedia)",2022,10,9,8
1,2022-10-09T08:45:38.483Z,"(ready, ecc907e1cd3cc9b88ac4, wikitext, None, ...",None,None,None,EditAttemptStep,None,"(Chrome, 106, None, Lenovo TB-7104F, False, Fa...",None,test.wikipedia.org,...,None,2022-10-09T08:45:45.137Z,"(None, None, None, None, {'user-agent': 'Mozil...","(None, 93c276f5-6e37-4c5d-a693-7d1820f6c8f7, e...","{'os_family': 'Android', 'browser_major': '106...","(wikipedia, test, [], org, wikipedia)",2022,10,9,8
2,2022-10-09T08:45:38.823Z,"(loaded, ecc907e1cd3cc9b88ac4, wikitext, None,...",None,None,None,EditAttemptStep,None,"(Chrome, 106, None, Lenovo TB-7104F, False, Fa...",None,test.wikipedia.org,...,None,2022-10-09T08:45:45.144Z,"(None, None, None, None, {'user-agent': 'Mozil...","(None, deec3806-b63d-42c9-9bb8-ef56c4935a5c, c...","{'os_family': 'Android', 'browser_major': '106...","(wikipedia, test, [], org, wikipedia)",2022,10,9,8


In [86]:
mea_query1=spark.run('''
SELECT 
  *
  FROM event.mediawiki_edit_attempt
  WHERE year = 2022
  AND month= 10
  AND day = 9
  AND hour = 8
  AND normalized_host.project = 'test'
  AND dt like '2022-10-09T08:45%'
   
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [87]:
mea_query1

,_schema,agent,custom_data,dt,http,mediawiki,meta,name,page,performer,user_agent_map,is_wmf_domain,normalized_host,datacenter,year,month,day,hour
0,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:14.517Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:11.643Z,...",eas.mf.save_intent,"(Requests/Permissions/SK fanella, None, 0, Non...","(None, 0, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
1,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:45.133Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:38.483Z,...",eas.mf.ready,"(Requests/Permissions/SK fanella, None, 146702...","(None, 1, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
2,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:17.384Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:11.645Z,...",eas.mf.save_attempt,"(Requests/Permissions/SK fanella, None, 0, Non...","(None, 0, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
3,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:18.690Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:11.656Z,...",eas.mf.save_success,"(Requests/Permissions/SK fanella, None, 0, Non...","(None, 0, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
4,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:44.516Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:38.458Z,...",eas.mf.init,"(Requests/Permissions/SK fanella, None, 146702...","(None, 1, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
5,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:45.139Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:38.813Z,...",eas.mf.loaded,"(Requests/Permissions/SK fanella, None, 146702...","(None, 1, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8


In [88]:
for v in mea_query1['user_agent_map']:
    print(v)

{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}
{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}
{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}
{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}
{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}
{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'b

In [82]:
for v in eas_query1['user_agent_map']:
    print(v)

{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}
{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}
{'os_family': 'Android', 'browser_major': '106', 'os_minor': '1', 'os_major': '8', 'wmf_app_version': '-', 'device_family': 'Lenovo TB-7104F', 'browser_family': 'Chrome'}


In [89]:
for v in mea_query1['performer']:
    print(v)

Row(can_probably_edit_page=None, edit_count=0, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='a034dd16df243ad238c2', registration_dt=None, session_id='0ace36c3e6f09d3668b0')
Row(can_probably_edit_page=None, edit_count=1, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='d72b7d1065bd6137d91f', registration_dt=None, session_id='0ace36c3e6f09d3668b0')
Row(can_probably_edit_page=None, edit_count=0, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='a034dd16df243ad238c2', registration_dt=None, session_id='0ace36c3e6f09d3668b0')
Row(can_probably_edit_page=None, edit_count=0, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='a034dd16

In [90]:
for v in eas_query1['event']:
    print(v)

Row(action='init', editing_session_id='ecc907e1cd3cc9b88ac4', editor_interface='wikitext', init_mechanism='click', init_type='section', integration='page', is_oversample=False, mw_version='1.40.0-wmf.4', page_id=146702, page_ns=4, page_title='Wikipedia:Requests/Permissions/SK_fanella', platform='phone', revision_id=547404, user_class=None, user_editcount=1, user_id=55333, version=1, save_failure_message=None, save_failure_type=None, abort_mechanism=None, abort_timing=None, abort_type=None, loaded_timing=None, page_token='d72b7d1065bd6137d91f', ready_timing=None, save_attempt_timing=None, save_intent_timing=None, save_success_timing=None, session_token='0ace36c3e6f09d3668b0', save_failure_timing=None, init_timing=None, bucket=None, anonymous_user_token=None, first_transaction_timing=None, first_change_timing=None)
Row(action='ready', editing_session_id='ecc907e1cd3cc9b88ac4', editor_interface='wikitext', init_mechanism=None, init_type=None, integration='page', is_oversample=False, mw_ve

In [91]:
eas_query2=spark.run('''
SELECT 
  *
  FROM event.editattemptstep
  WHERE 
  event.session_token='0ace36c3e6f09d3668b0'
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [92]:
eas_query2

,dt,event,ip,recvfrom,revision,schema,seqid,useragent,uuid,webhost,...,_schema,client_dt,http,meta,user_agent_map,normalized_host,year,month,day,hour
0,2022-10-09T08:45:38.473Z,"(init, ecc907e1cd3cc9b88ac4, wikitext, click, ...",None,None,None,EditAttemptStep,None,"(Chrome, 106, None, Lenovo TB-7104F, False, Fa...",None,test.wikipedia.org,...,None,2022-10-09T08:45:44.524Z,"(None, None, None, None, {'user-agent': 'Mozil...","(None, 195de194-b0e5-4239-81e2-d9e99ac6f282, 2...","{'os_family': 'Android', 'browser_major': '106...","(wikipedia, test, [], org, wikipedia)",2022,10,9,8
1,2022-10-09T08:45:38.483Z,"(ready, ecc907e1cd3cc9b88ac4, wikitext, None, ...",None,None,None,EditAttemptStep,None,"(Chrome, 106, None, Lenovo TB-7104F, False, Fa...",None,test.wikipedia.org,...,None,2022-10-09T08:45:45.137Z,"(None, None, None, None, {'user-agent': 'Mozil...","(None, 93c276f5-6e37-4c5d-a693-7d1820f6c8f7, e...","{'os_family': 'Android', 'browser_major': '106...","(wikipedia, test, [], org, wikipedia)",2022,10,9,8
2,2022-10-09T08:45:38.823Z,"(loaded, ecc907e1cd3cc9b88ac4, wikitext, None,...",None,None,None,EditAttemptStep,None,"(Chrome, 106, None, Lenovo TB-7104F, False, Fa...",None,test.wikipedia.org,...,None,2022-10-09T08:45:45.144Z,"(None, None, None, None, {'user-agent': 'Mozil...","(None, deec3806-b63d-42c9-9bb8-ef56c4935a5c, c...","{'os_family': 'Android', 'browser_major': '106...","(wikipedia, test, [], org, wikipedia)",2022,10,9,8


In [106]:
mea_query2=spark.run('''
SELECT 
  *
  FROM event.mediawiki_edit_attempt
  WHERE 
  performer.session_id='0ace36c3e6f09d3668b0'
   
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [107]:
mea_query2

,_schema,agent,custom_data,dt,http,mediawiki,meta,name,page,performer,user_agent_map,is_wmf_domain,normalized_host,datacenter,year,month,day,hour
0,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:54:15.046Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:54:08.784Z,...",eas.mf.init,"(Requests/Permissions/SK fanella, None, 146702...","(None, 2, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
1,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:53:24.471Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:53:46.296Z,...",eas.mf.ready,"(Requests/Permissions/SK fanella, None, 146702...","(None, 1, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
2,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:14.517Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:11.643Z,...",eas.mf.save_intent,"(Requests/Permissions/SK fanella, None, 0, Non...","(None, 0, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
3,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:54:16.158Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:54:09.076Z,...",eas.mf.loaded,"(Requests/Permissions/SK fanella, None, 146702...","(None, 2, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
4,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:45:45.133Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:45:38.483Z,...",eas.mf.ready,"(Requests/Permissions/SK fanella, None, 146702...","(None, 1, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
5,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:53:37.461Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:53:46.308Z,...",eas.mf.first_change,"(Requests/Permissions/SK fanella, None, 146702...","(None, 1, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
6,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:53:51.660Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-09T08:53:46.427Z,...",eas.mf.save_intent,"(Requests/Permissions/SK fanella, None, 146702...","(None, 1, None, None, 55333, None, True, None,...","{'os_family': 'Android', 'browser_major': '106...",True,"(wikipedia, test, [], org, wikipedia)",eqiad,2022,10,9,8
7,None,"(None, mediawiki_js, mobile_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-10-09T08:53:53.656Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.4, N...","(test.wikipedia.org, 2022-10-0

In [108]:
for v in mea_query2['performer']:
    print(v)

Row(can_probably_edit_page=None, edit_count=2, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='77110f6e25d1330cde65', registration_dt=None, session_id='0ace36c3e6f09d3668b0')
Row(can_probably_edit_page=None, edit_count=1, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='49d20bc7abf148c2b22f', registration_dt=None, session_id='0ace36c3e6f09d3668b0')
Row(can_probably_edit_page=None, edit_count=0, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='a034dd16df243ad238c2', registration_dt=None, session_id='0ace36c3e6f09d3668b0')
Row(can_probably_edit_page=None, edit_count=2, edit_count_bucket=None, groups=None, id=55333, is_bot=None, is_logged_in=True, language=None, language_variant=None, name=None, pageview_id='77110f6e

In [97]:
for v in eas_query2['event']:
    print(v)

Row(action='init', editing_session_id='ecc907e1cd3cc9b88ac4', editor_interface='wikitext', init_mechanism='click', init_type='section', integration='page', is_oversample=False, mw_version='1.40.0-wmf.4', page_id=146702, page_ns=4, page_title='Wikipedia:Requests/Permissions/SK_fanella', platform='phone', revision_id=547404, user_class=None, user_editcount=1, user_id=55333, version=1, save_failure_message=None, save_failure_type=None, abort_mechanism=None, abort_timing=None, abort_type=None, loaded_timing=None, page_token='d72b7d1065bd6137d91f', ready_timing=None, save_attempt_timing=None, save_intent_timing=None, save_success_timing=None, session_token='0ace36c3e6f09d3668b0', save_failure_timing=None, init_timing=None, bucket=None, anonymous_user_token=None, first_transaction_timing=None, first_change_timing=None)
Row(action='ready', editing_session_id='ecc907e1cd3cc9b88ac4', editor_interface='wikitext', init_mechanism=None, init_type=None, integration='page', is_oversample=False, mw_ve

In [109]:
for v in mea_query2['custom_data']:
    print(v)

{'integration': Row(data_type='string', value='page'), 'editing_session_id': Row(data_type='string', value='860dad740e96c94289af'), 'init_mechanism': Row(data_type='string', value='click'), 'editor_interface': Row(data_type='string', value='wikitext'), 'init_type': Row(data_type='string', value='section')}
{'integration': Row(data_type='string', value='page'), 'editing_session_id': Row(data_type='string', value='44823d21f625c863551a'), 'editor_interface': Row(data_type='string', value='wikitext'), 'ready_timing': Row(data_type='number', value='769')}
{'integration': Row(data_type='string', value='page'), 'editing_session_id': Row(data_type='string', value='15afe7e51c2802fc7204'), 'save_intent_timing': Row(data_type='number', value='31928'), 'editor_interface': Row(data_type='string', value='wikitext')}
{'integration': Row(data_type='string', value='page'), 'editing_session_id': Row(data_type='string', value='860dad740e96c94289af'), 'editor_interface': Row(data_type='string', value='wik

In [ ]:
In this example it seems like 

In [72]:
eas_query1=spark.run('''
SELECT 
  --date_format(dt, 'yyyy-MM-dd') AS date, 
  date_format(dt, 'yyyy-MM') AS date, 
  count(1) AS events
  FROM event.editattemptstep
  WHERE 
  year = 2022 
  AND wiki = 'testwiki' 
  GROUP BY date 
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [73]:
eas_query1.sort_values(by=['date'],ascending=False)

,date,events
2,2022-11,2424
0,2022-10,1243
1,2022-09,1529
3,2022-08,1324


In [68]:
mea_query1=spark.run('''
SELECT 
  date_format(dt, 'yyyy-MM') AS date,
  count(1) AS events
  FROM event.mediawiki_edit_attempt
  WHERE 
  year = 2022
  AND normalized_host.project = 'test'
  GROUP BY date 
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [71]:
mea_query1.sort_values(by=['date'],ascending=False)

,date,events
2,2022-11,497
0,2022-10,1189
1,2022-09,1120
